# Preliminary

In [1]:
words = ['this', 'the', 'these', 'to', 'toe', 'cat']

In [2]:
from selkie.fst import Fst
fst = Fst()

In [3]:
fst.edge(0, 2, 't')
fst.edge(2, 3, 'h')
fst.edge(3, 4, 'i')
fst.edge(4, 1, 's', 'this')

<Edge 4 1 s this>

In [4]:
fst.dump()

Fst:
  ->0  [0]
    1  [2]
    2  [3]
    3  [4]
    4  [1]
    0 1 t :
    1 2 h :
    2 3 i :
    3 4 s : this


In [5]:
q = fst.start
q['t']

[<Edge 0 2 t None>]

In [6]:
e = q['t'][0]
(e.dest, e.inlabel, e.outlabel)

(<State 2 [1]>, 't', None)

In [7]:
q['c']

[]

In [8]:
len(fst)

5

# The advance() function

In [9]:
def advance (fst, q, i):
    edges = q[i]
    if len(edges) == 0:
        r = len(fst)
        print('create edge', q, r, i)
        return fst.edge(q, r, i)
    elif len(edges) == 1:
        return edges[0]
    else:
        raise Exception(f'Non-deterministic state: {q} {i}')

In [10]:
q = fst.start
e = advance(fst, q, 't')
q = e.dest
e = advance(fst, q, 'h')
q = e.dest
e = advance(fst, q, 'e')
e.outlabel = 'the'
fst.dump()

create edge 3 5 e
Fst:
  ->0  [0]
    1  [2]
    2  [3]
    3  [4]
    4  [1]
    5  [5]
    0 1 t :
    1 2 h :
    2 3 i :
    2 5 e : the
    3 4 s : this


# The convert() function

In [11]:
def convert (wordlist):
    fst = Fst()
    fst.start = start = fst.state(0)
    for word in wordlist:
        q = start
        for c in word:
            e = advance(fst, q, c)
            q = e.dest
        if e.outlabel and e.outlabel != word:
            raise Exception(f'Attempt to change output word: {e.source} {i} {word}, was {e.outlabel}')
        e.outlabel = word
        fst.final_state(q)
    return fst

In [12]:
fst = convert(words)
fst.dump()

create edge 0 1 t
create edge 1 2 h
create edge 2 3 i
create edge 3 4 s
create edge 2 5 e
create edge 5 6 s
create edge 6 7 e
create edge 1 8 o
create edge 8 9 e
create edge 0 10 c
create edge 10 11 a
create edge 11 12 t
Fst:
  ->0  [0]
    1  [1]
    2  [2]
    3  [3]
    4# [4]
    5# [5]
    6  [6]
    7# [7]
    8# [8]
    9# [9]
    10  [10]
    11  [11]
    12# [12]
    0 1 t :
    0 10 c :
    1 2 h :
    1 8 o : to
    2 3 i :
    2 5 e : the
    3 4 s : this
    5 6 s :
    6 7 e : these
    8 9 e : toe
    10 11 a :
    11 12 t : cat


In [13]:
list(fst)

[(['c', 'a', 't'], ['cat']),
 (['t', 'o'], ['to']),
 (['t', 'o', 'e'], ['to', 'toe']),
 (['t', 'h', 'e'], ['the']),
 (['t', 'h', 'e', 's', 'e'], ['the', 'these']),
 (['t', 'h', 'i', 's'], ['this'])]

# fst_from_list()

In [14]:
from selkie.fst import from_list
fst = from_list(words)
fst.dump()

Fst:
  ->0  [0]
    1  [1]
    2  [2]
    3  [3]
    4# [4]
    5# [5]
    6  [6]
    7# [7]
    8# [8]
    9# [9]
    10  [10]
    11  [11]
    12# [12]
    0 1 t :
    0 10 c :
    1 2 h :
    1 8 o : to
    2 3 i :
    2 5 e : the
    3 4 s : this
    5 6 s :
    6 7 e : these
    8 9 e : toe
    10 11 a :
    11 12 t : cat


In [15]:
from selkie.io import redirect
with redirect():
    fst.dump()
print(repr(redirect.output))

'Fst:\n  ->0  [0]\n    1  [1]\n    2  [2]\n    3  [3]\n    4# [4]\n    5# [5]\n    6  [6]\n    7# [7]\n    8# [8]\n    9# [9]\n    10  [10]\n    11  [11]\n    12# [12]\n    0 1 t :\n    0 10 c :\n    1 2 h :\n    1 8 o : to\n    2 3 i :\n    2 5 e : the\n    3 4 s : this\n    5 6 s :\n    6 7 e : these\n    8 9 e : toe\n    10 11 a :\n    11 12 t : cat\n'


In [16]:
list(fst)

[(['c', 'a', 't'], ['cat']),
 (['t', 'o'], ['to']),
 (['t', 'o', 'e'], ['to', 'toe']),
 (['t', 'h', 'e'], ['the']),
 (['t', 'h', 'e', 's', 'e'], ['the', 'these']),
 (['t', 'h', 'i', 's'], ['this'])]

In [17]:
sorted(o[-1] for (i, o) in fst)

['cat', 'the', 'these', 'this', 'to', 'toe']